In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

print(sys.path)

['/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/scur0279/MMA/.venv/lib64/python3.9/site-packages', '/home/scur0279/MMA/.venv/lib/python3.9/site-packages', '/gpfs/home5/scur0279/MMA/app']


In [2]:
from db.database import Database
import uuid

/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scur0279/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/scur0279/MMA/.venv/lib64/python3.9/site-packages/lpips/weights/v0.1/alex.pth


In [3]:
ls

create.py    draft_database.py  gilian_database.py  images/      mma.db
database.py  example.ipynb      gilian_mmb.db       __init__.py  __pycache__/


#### stuff for testing the saving

In [4]:
from modules.prompt import Prompt
from modules.prompt_image import PromptImage
# from db.database import Database
from PIL import Image

# -------------------
# DB SETUP
# -------------------
db = Database()
db.connect()
db.reset_tables()
print("[DB RESET]")

# -------------------
# USERS
# -------------------
user_id_1 = db.insert_user("first_test_user", "password123")
user_id_2 = db.insert_user("second_test_user", "password456")
user_id_1 = 1
user_id_2 = 2
print(f"[USERS] user_1={user_id_1}, user_2={user_id_2}")

# -------------------
# CHAT 1: Simple perfume chain (user 1)
# -------------------
chat_1 = db.insert_chat("Perfume Prompt Chain", user_id_1)
print(f"[CHAT 1] id={chat_1}")

prompt1 = Prompt("Generate a bottle of perfume", depth=1)
prompt2 = Prompt("Generate a bottle of perfume in a desert", depth=2)
prompt3 = Prompt("Add sunset lighting to the perfume bottle scene", depth=3,
                 suggestion_used="Try sunset colors", modified_suggestion=False)
prompt3.is_enhanced = True
prompt3.enhanced_prompt = "A perfume bottle in a desert with glowing sunset hues"

for p in [prompt1, prompt2, prompt3]:
    db.save_prompt(p, chat_1, user_id=user_id_1)

image_paths = ["images/sauvage.png", "images/sauvage_desert.png", "images/sauvage_sunset.png"]
for path, prompt in zip(image_paths, [prompt1, prompt2, prompt3]):
    img = Image.open(path)
    pi = PromptImage(image=img, prompt_guidance=6.0, image_guidance=0.4,
                     input_prompt=prompt.id, output_prompt=prompt.id, save=True, selected=True)
    db.save_image(pi, session_id=chat_1, user_id=user_id_1)

db.connect()
# -------------------
# CHAT 2: Cyberpunk drone (user 1)
# -------------------
chat_2 = db.insert_chat("Futuristic Drone Chain", user_id_1)
print(f"[CHAT 2] id={chat_2}")

drone1 = Prompt("Design a futuristic drone", depth=1)
drone2 = Prompt("Add neon lighting to the drone", depth=2)
shared_drone_prompt = Prompt("Place the drone in a cyberpunk city", depth=3,
                             suggestion_used="Try placing it in a city", modified_suggestion=True)

# Save prompts
for p in [drone1, drone2, shared_drone_prompt]:
    db.save_prompt(p, chat_2, user_id=user_id_1)

# Image 3: from shared prompt, not selected
img3 = Image.open("images/test_image3.jpg")
pi3 = PromptImage(image=img3, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=shared_drone_prompt.id, output_prompt=shared_drone_prompt.id, save=True)
db.save_image(pi3, session_id=chat_2, user_id=user_id_1)

# Image 4: from same shared prompt, selected
img4 = Image.open("images/test_image4.jpg")
pi4 = PromptImage(image=img4, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=shared_drone_prompt.id, output_prompt=shared_drone_prompt.id, save=True, selected=True)
# pi4.selected = True
db.save_image(pi4, session_id=chat_2, user_id=user_id_1)

# Add drone1 and drone2 images (selected)
img1 = Image.open("images/test_image.jpg")
pi1 = PromptImage(image=img1, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=drone1.id, output_prompt=drone1.id, save=True, selected=True)
# pi1.selected = True
db.save_image(pi1, session_id=chat_2, user_id=user_id_1)

img2 = Image.open("images/test_image2.jpg")
pi2 = PromptImage(image=img2, prompt_guidance=7.0, image_guidance=0.5,
                  input_prompt=drone2.id, output_prompt=drone2.id, save=True, selected=True)
# pi2.selected = True
db.save_image(pi2, session_id=chat_2, user_id=user_id_1)

# -------------------
# CHAT 3: Deep sci-fi chain (user 2)
# -------------------
db.connect()
chat_3 = db.insert_chat("Sci-Fi Fleet Chain", user_id_2)
print(f"[CHAT 3] id={chat_3}")

p1 = Prompt("A spaceship flying through clouds", depth=1)
p2 = Prompt("Add sunlight breaking through clouds", depth=2)
p3 = Prompt("Include a fleet of ships in the distance", depth=3)
p4 = Prompt("Make the ships look metallic", depth=4,
            suggestion_used="Make them metallic", modified_suggestion=False)
p5 = Prompt("Add reflections from the sun", depth=5)
p5.is_enhanced = True
p5.enhanced_prompt = "Fleet of metallic ships reflecting golden sunlight"

for p in [p1, p2, p3, p4, p5]:
    db.save_prompt(p, chat_3, user_id=user_id_2)

sci_images = [
    "images/cat.png",
    "images/cat1.png",
    "images/cat2.png",
    "images/cat3.png",
    "images/cat4.png"
]
guidances = [(6.0, 0.4), (6.5, 0.45), (7.0, 0.5), (7.5, 0.55), (8.0, 0.6)]

for path, (pg, ig), prompt in zip(sci_images, guidances, [p1, p2, p3, p4, p5]):
    img = Image.open(path)
    pi = PromptImage(image=img, prompt_guidance=pg, image_guidance=ig,
                     input_prompt=prompt.id, output_prompt=prompt.id, save=True, selected=True)
    pi.selected = True
    db.save_image(pi, session_id=chat_3, user_id=user_id_2)

# -------------------
# Done
# -------------------
db.close()
print("[DONE] Data seeded.")


[DB RESET]
[USERS] user_1=1, user_2=2
[CHAT 1] id=1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[CHAT 2] id=2


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[CHAT 3] id=3


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

[DONE] Data seeded.


In [7]:
#### Test all the fetch methods

chat_id = 1

def test_fetch_methods():
    db = Database()
    db.connect()

    print("\n========== USERS ==========")
    print("All users:", db.fetch_all_users().shape)
    print("User by ID (1):\n", db.fetch_user_by_id(1))
    print("User by Username:\n", db.fetch_user_by_username("second_test_user"))

    print("\n========== CHATS ==========")
    print("All chats:", db.fetch_all_chats().shape)
    print("Chats by User (1):\n", db.fetch_chats_by_user(1))
    print("Chat by ID (1):\n", db.fetch_chat_by_id(1))

    print("\n========== PROMPTS ==========")
    print("All prompts:", db.fetch_all_prompts().shape)
    print("Prompts by Chat (1):\n", db.fetch_prompts_by_chat(chat_id))
    print("Prompts by User (1):\n", db.fetch_prompts_by_user(1))

    # Use a known prompt ID from your inserted test data
    print("Prompt by ID:\n", db.fetch_prompt_by_id("1"))  # Replace "1" with real prompt id if needed

    print("\n========== IMAGES ==========")
    print("All images:", db.fetch_all_images().shape)
    print("Images by User (1):\n", db.fetch_images_by_user(1))
    print("Images by Chat (1):\n", db.fetch_images_by_chat(chat_id))
    # doesn't work because need the uuid path name for the image
    print("Image by Path (images/e1a3fe54-f85d-4e86-a86f-9d076032f2e2.png):\n", db.fetch_image_by_path("images/e1a3fe54-f85d-4e86-a86f-9d076032f2e2.png"))

    # Replace with actual image_id from your test database
    sample_image_id = db.fetch_all_images().iloc[0]["id"]
    print(f"Image by ID ({sample_image_id}):\n", db.fetch_image_by_id(sample_image_id))

    print("\n========== METRICS ==========")

    print("All BERTScore metrics:\n", db.fetch_all_bertscore_metrics().shape)
    print("BERTScore by User (1):\n", db.fetch_bertscore_by_user(1))
    print("BERTScore by Chat (1):\n", db.fetch_bertscore_by_chat(chat_id))

    print("\nGuidance metrics:", db.fetch_all_guidance_metrics().shape)
    print("Guidance by User (1):\n", db.fetch_guidance_by_user(1))

    print("\nLPIPS metrics:", db.fetch_all_lpips_metrics().shape)
    print("LPIPS by Chat (1):\n", db.fetch_lpips_by_chat(chat_id))

    print("\nFunctionality metrics:", db.fetch_all_functionality_metrics().shape)
    print("Functionality by Chat (1):\n", db.fetch_functionality_by_chat(chat_id))

    print("\nPrompt Word metrics:", db.fetch_all_prompt_word_metrics().shape)
    print("Prompt Word by Chat (1):\n", db.fetch_prompt_word_metrics_by_chat(1))

    db.close()

# Run the tests
test_fetch_methods()


========== USERS ==========
All users: (2, 3)
User by ID (1):
    id         username     password
0   1  first_test_user  password123
User by Username:
    id          username     password
0   2  second_test_user  password456

========== CHATS ==========
All chats: (3, 3)
Chats by User (1):
    id                   title  user_id
0   1    Perfume Prompt Chain        1
1   2  Futuristic Drone Chain        1
Chat by ID (1):
    id                 title  user_id
0   1  Perfume Prompt Chain        1

========== PROMPTS ==========
All prompts: (11, 12)
Prompts by Chat (1):
                                      id  user_id  chat_id  \
0  4fc8c199-c6df-4578-b8b2-302722da69a7        1        1   
1  78e321aa-1c3e-4e37-8be6-a8cadb5b1312        1        1   
2  9ae88630-bb20-417e-a782-0867c5a6ce4a        1        1   

                                            prompt  depth  used_suggestion  \
0                     Generate a bottle of perfume      1                0   
1         Generate a

In [ ]:
datab = Database()
datab.connect()

In [ ]:
df_bert = datab.fetch_dataframe("SELECT * FROM bertscore_metrics")
df_guidance = datab.fetch_dataframe("SELECT * FROM guidance_metrics")
df_lpips = datab.fetch_dataframe("SELECT * FROM lpips_metrics")
df_func = datab.fetch_dataframe("SELECT * FROM functionality_metrics")
df_words = datab.fetch_dataframe("SELECT * FROM prompt_word_metrics")

print("BERTScore Metrics:\n", df_bert)
print("Guidance Metrics:\n", df_guidance)
print("LPIPS Metrics:\n", df_lpips)
print("Functionality Metrics:\n", df_func)
print("Prompt Word Metrics:\n", df_words)


### MMA Metrics Data Cheatsheet

Please note: two different users cannot have the same chat_id in your current database design.

```
CREATE TABLE IF NOT EXISTS chats (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    user_id INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
```

the id column (chat_id) is a unique, auto-incrementing primary key. Therefore, each chat_id is globally unique and tied to a single user_id. It's impossible for the same chat_id to belong to more than one user.

#### Setup

Make sure you have imported and connected:

```python
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

print(sys.path)
from db.database import Database
import uuid

db = Database()
db.connect()
```

---

#### Metric Overview

| Metric Type       | Table Name              | What It Measures                                    |
| ----------------- | ----------------------- | --------------------------------------------------- |
| **BERTScore**     | `bertscore_metrics`     | Novelty of current prompt vs previous               |
| **Guidance**      | `guidance_metrics`      | Prompt/image guidance values used during generation |
| **LPIPS**         | `lpips_metrics`         | Visual change between images                        |
| **Functionality** | `functionality_metrics` | Suggestion/enhancement usage stats                  |
| **Prompt Words**  | `prompt_word_metrics`   | Word-level analysis of prompts                      |

---

#### Fetch All Metrics

```python
df_bert = db.fetch_all_bertscore_metrics()
df_guidance = db.fetch_all_guidance_metrics()
df_lpips = db.fetch_all_lpips_metrics()
df_func = db.fetch_all_functionality_metrics()
df_words = db.fetch_all_prompt_word_metrics()
```

In [ ]:
datab.fetch_all_bertscore_metrics()

In [ ]:
datab.fetch_all_guidance_metrics()

In [ ]:
datab.fetch_all_lpips_metrics()

In [ ]:
from modules.metrics import calculate_lpips

score = calculate_lpips("images/cat1.png", "images/cat2.png")
print("Manual LPIPS:", score)

In [ ]:

datab.fetch_all_functionality_metrics()

In [ ]:

datab.fetch_all_prompt_word_metrics()

---

#### Fetch by User

```python
user_id = 1

df_bert = db.fetch_bertscore_by_user(user_id)
df_guidance = db.fetch_guidance_by_user(user_id)
df_lpips = db.fetch_lpips_by_user(user_id)
df_func = db.fetch_functionality_by_user(user_id)
df_words = db.fetch_prompt_word_metrics_by_user(user_id)
```

In [ ]:
user_id = 1

datab.fetch_bertscore_by_user(user_id)

In [ ]:
datab.fetch_guidance_by_user(user_id)

In [ ]:
user_id = 1
datab.fetch_lpips_by_user(user_id)

In [ ]:
datab.fetch_functionality_by_user(user_id)

In [ ]:
datab.fetch_prompt_word_metrics_by_user(user_id)

---

#### Fetch by Chat

```python
chat_id = 2

df_bert = db.fetch_bertscore_by_chat(chat_id)
df_guidance = db.fetch_guidance_by_chat(chat_id)
df_lpips = db.fetch_lpips_by_chat(chat_id)
df_func = db.fetch_functionality_by_chat(chat_id)
df_words = db.fetch_prompt_word_metrics_by_chat(chat_id)
```



In [ ]:
chat_id = 1

datab.fetch_bertscore_by_chat(chat_id)

In [ ]:
datab.fetch_guidance_by_chat(chat_id)

In [ ]:
chat_id = 1
datab.fetch_lpips_by_chat(chat_id)

In [ ]:
datab.fetch_functionality_by_chat(chat_id)

In [ ]:
datab.fetch_functionality_by_chat(3)

In [ ]:
datab.fetch_prompt_word_metrics_by_chat(chat_id)

---

#### Column Hints

##### `bertscore_metrics`

* `bert_novelty`: float (0–1)
* `prompt_id`, `previous_prompt_id`

##### `guidance_metrics`

* `prompt_guidance`, `image_guidance`: floats
* `depth`, `prompt_id`

##### `lpips_metrics`

* `lpips`: float (0–1, lower = more similar)
* `image_id`, `previous_image_id`

##### `functionality_metrics`

* `used_suggestion_pct`, `used_enhancement_pct`, `used_both_pct`, `no_ai_pct`: % as floats
* One row per (user, chat)

##### `prompt_word_metrics`

* `full_text`: full prompt
* `relevant_words`: comma-separated words
* `word_count`: number of words after stopword removal


#### stuff for testing the inserting (insert_*)

In [ ]:
datab = Database()
datab.connect()

In [ ]:
# Clears all previous database entries
datab.reset_tables()

In [ ]:
# --- Step 1: Create Users ---
datab.insert_user("first_test_user", "password123")
datab.insert_user("second_test_user", "password456")

In [ ]:
# --- Step 2: Create Chats ---
datab.insert_chat("First Test Chat", 1)
datab.insert_chat("Second Test Chat", 2)

chat_id_1=1
chat_id_2=2

# --- Step 3: Create Prompts ---
prompt_id_0 = str(uuid.uuid4())
prompt_id_1 = str(uuid.uuid4())
prompt_id_2 = str(uuid.uuid4())
prompt_id_3 = str(uuid.uuid4())
prompt_id_4 = str(uuid.uuid4())

# Inputs for insert_prompt function:
# prompt_id, chat_id, user_id, prompt, depth, used_suggestion=False, modified_suggestion=False, suggestion_used=None, is_enhanced=False, enhanced_prompt=None, image_in_id=None, images_out=None

# note: as it stands Gilian's design expects each prompt_id to be a globally unique ID, such as a UUID.
# this is because the code says "CREATE TABLE IF NOT EXISTS prompts (id TEXT PRIMARY KEY,...)"
# we can maybe create a composite key (e.g., PRIMARY KEY (chat_id, id)) so that we distinguish them by chat/user
datab.insert_prompt(
    id=prompt_id_0, chat_id=chat_id_1, user_id=1,
    prompt="Generate a bottle of perfume", depth=1
)

datab.insert_prompt(
    id=prompt_id_1, chat_id=chat_id_1, user_id=1,
    prompt="Generate a bottle of perfume in a desert", depth=2
)

datab.insert_prompt(
    id=prompt_id_2, chat_id=chat_id_1, user_id=1,
    prompt="Add sunset lighting to the perfume bottle scene", depth=3,
    used_suggestion=True, suggestion_used="Try sunset colors", is_enhanced=True,
    enhanced_prompt="A perfume bottle in a desert with glowing sunset hues"
)

datab.insert_prompt(
    id=prompt_id_3, chat_id=chat_id_2, user_id=2,
    prompt="Design a stylish water bottle sticker", depth=1,
    used_suggestion=True, modified_suggestion=True, suggestion_used="Urban aesthetic",
    is_enhanced=True, enhanced_prompt="Water bottle sticker with urban graffiti"
)

datab.insert_prompt(
    id=prompt_id_4, chat_id=chat_id_2, user_id=2,
    prompt="Make it into a metallic sticker", depth=2,
    is_enhanced=False
)

# As can be seen in the code below, the input_prompt_id cannot be a str
# if not isinstance(input_prompt_id, str):
#     raise ValueError("input_prompt_id must be an integer")

# --- Step 4: Create Images ---
image_id_1 = str(uuid.uuid4())
image_id_2 = str(uuid.uuid4())
image_id_3 = str(uuid.uuid4())
image_id_4 = str(uuid.uuid4())

# Inputs for insert_image function:
# image_id: str, user_id: int, chat_id: int, prompt_guidance: float, image_guidance: float, path: str, input_prompt_id: str, output_prompt_id: str,
datab.insert_image(
    id=image_id_1, user_id=1, chat_id=chat_id_1,
    prompt_guidance=6.0, image_guidance=0.4,
    path="app/db/images/sauvage.png",
    input_prompt_id=prompt_id_1, output_prompt_id=prompt_id_1
)

datab.insert_image(
    id=image_id_2, user_id=1, chat_id=chat_id_1,
    prompt_guidance=7.0, image_guidance=0.5,
    path="app/db/images/sauvage_desert.png",
    input_prompt_id=prompt_id_2, output_prompt_id=prompt_id_2
)

datab.insert_image(
    id=image_id_3, user_id=2, chat_id=chat_id_2,
    prompt_guidance=5.5, image_guidance=0.3,
    path="app/db/images/test_image2.jpg",
    input_prompt_id=prompt_id_3, output_prompt_id=prompt_id_3
)

datab.insert_image(
    id=image_id_4, user_id=2, chat_id=chat_id_2,
    prompt_guidance=6.2, image_guidance=0.2,
    path="app/db/images/test_image4.jpg",
    input_prompt_id=prompt_id_4, output_prompt_id=prompt_id_4
)

In [ ]:
with Database() as data_base:  # adjust path if needed
    print("=== USERS ===")
    print(data_base.fetch_all_users())
    
    print("=== CHATS ===")
    print(data_base.fetch_all_chats())
    
    print("=== PROMPTS ===")
    print(data_base.fetch_all_prompts())
    
    print("=== IMAGES ===")
    print(data_base.fetch_all_images())

In [ ]:
data_base = Database()
data_base.connect()

In [ ]:
current_prompt_df = data_base.fetch_prompt_by_id('8dd6a2f4-6622-4a63-a093-8eac09b07b0b', pandas=True)

In [ ]:
if not current_prompt_df.empty:
    depth = current_prompt_df.iloc[0]["depth"]
else:
    depth = 0

print(depth)

In [ ]:
prior_images = data_base.fetch_images_by_chat(chat_id_1, pandas=True)
# print(f"prior_images1: {prior_images}")
prior_images = prior_images.merge(data_base.fetch_all_prompts(), how="left", left_on="output_prompt_id", right_on="id")
# print(f"prior_images2: {prior_images}")
prior_images = prior_images[prior_images["depth"] == depth - 1]
print(f"prior_images3: {prior_images}")

In [ ]:
previous_image_id = None
previous_image_path = None
if not prior_images.empty:
        previous_image_id = prior_images.iloc[0]["id_x"]  # id_x = image.id
        print(previous_image_id)
        previous_image_path = prior_images.iloc[0]["path"]
        print(previous_image_path)


In [ ]:
current_images = data_base.fetch_images_by_chat(chat_id_1, pandas=True)
# print(f"prior_images1: {prior_images}")
current_images = current_images.merge(data_base.fetch_all_prompts(), how="left", left_on="output_prompt_id", right_on="id")
current_images = current_images[current_images["depth"] == depth]
print(f"current_images3: {current_images}")
print(current_images.iloc[0]["id_x"])
print(current_images.iloc[0]["path"])

In [ ]:
from modules.metrics import get_or_compute_bertscore, extract_relevant_words, get_or_compute_lpips

In [ ]:
lpips_score = None
if previous_image_path:
    lpips_score = get_or_compute_lpips("6122bcfc-fb9d-4f2c-afdf-bd93192b4b22", "images/sauvage_desert.png", "images/sauvage.png")
else:
    lpips_score = None

print(str(lpips_score))

In [ ]:
ls

In [ ]:
data_base = Database()
data_base.connect()

In [ ]:

datab.fetch_all_prompts()

In [ ]:
# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

datab.fetch_all_users(pandas=True)

In [ ]:

# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

datab.fetch_all_images(pandas=True)

In [ ]:
datab.close()